<a href="https://colab.research.google.com/github/AnitaKirkovska/MachineLearning_HW1/blob/master/Problem3_DifferentApproach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.datasets import mnist
import numpy as np
from keras import backend as K
import keras
from keras.utils import to_categorical

Using TensorFlow backend.


I had challenges doing the third problem with the same model as in the book, so I have used a help from a colleague as we solved it together so the style is different for this one. But this one works.

In [0]:
(x_train_org, y_train_org),(x_test_org, y_test_org) = mnist.load_data()

In [0]:
x_train = x_train_org.reshape((60000, 28 * 28))
x_train = x_train_org.astype('float32') / 255

x_test = x_test_org.reshape((10000, 28 * 28))
x_test = x_test_org.astype('float32') / 255

10 Classes with categorical labels

In [0]:
x_train = x_train.reshape(x_train.shape[0], -1)
x_test = x_test.reshape(x_test.shape[0], -1)
y_train = to_categorical(y_train_org, num_classes=10) 
y_test = to_categorical(y_test_org, num_classes=10)


In [0]:
def update_links(w,b,X):
  return np.dot(X,w.T) + b
def softmax(links):
  exp = np.exp(links)
  sum_exp = np.sum(np.exp(links), axis=1, keepdims=True)
  softmax = exp / sum_exp
  return softmax
def cross_entropy(X, Y, links):
  m = X.shape[0]
  loss = - (1 / m) * np.sum(Y * np.log(links))
  return loss

Transform vector y of labels to one-hot encoded matrix

In [0]:
def one_hot(X, Y, num_class=10):
  m = X.shape[0]
  one_hot = np.zeros((m, num_class))
  one_hot[np.arange(m), Y.T] = 1
  return one_hot

Weight and bias parameters

In [0]:
def params_zeros(num_class, num_features):
  w = np.random.rand(num_class, num_features)
  b = np.zeros((1, num_class))
  return w,b

Prediction

In [0]:
def predict(w,b,X):
  links = update_links(w,b,X)
  probs = softmax(links)
  preds = np.argmax(probs,axis=1)

  return preds

Training

In [0]:
def train(X_train, Y_train, batch_size=10, lr=0.01,num_class = 10):
  train_samples,train_features = X_train.shape
  w, b = params_zeros(num_class,train_features)
  #print (w,b)
  #print (train_samples, train_features)
  #print(w.shape)
  all_losses = []
  
  #stochastic gradient 
  for item in range(batch_size):
    cost = 0.0
    for i in range(train_samples):
      
      rand_ind = np.random.randint(0,train_samples)
      xi = X_train[rand_ind,:].reshape(1,train_features)
      yi = Y_train[rand_ind].reshape(1,1)
      
      y_one_hot = one_hot(xi,yi)
      scores = update_links(w,b,xi)
      probs = softmax(scores)
      loss = cross_entropy(xi,y_one_hot, probs)
      
      #print(probs.shape)
      dw = (1. / train_samples) * np.dot((probs - y_one_hot).T,xi)
      db = (1. / train_samples) * np.sum(probs - y_one_hot, axis=0)
      
      w = w - lr * dw
      b = b - lr * db
      
    all_losses.append(loss)
    
    print(f'Iteration number: {item}, loss: {np.round(loss, 4)}')
    
  return w, b, all_losses

Model

In [0]:
def model(X_train, Y_train, X_test, Y_test, batch_size, lr,num_class):
  w, b, loss = train(X_train, Y_train, batch_size, lr, num_class)
  
  #predict test/train set examples 
  Y_pred_test = predict(w, b, X_test)
  Y_pred_train = predict(w, b, X_train)
 
  Y_true_test = np.argmax(Y_test, axis=0)
  Y_true_train = np.argmax(Y_train,axis=0)
                         

  print("")
  print("train accuracy: {} %".format(100*sum(Y_pred_train==Y_true_train)/(float(len(Y_train)))))
  print("test accuracy: {} %".format(100*sum(Y_pred_test==Y_true_test)/(float(len(Y_test)))))
 
  d = {"costs": loss, "Y_pred_test": Y_pred_test, 
        "Y_pred_train" : Y_pred_train, 
        "w" : w, 
        "b" : b,
        "learning_rate" : lr,
        "batch_size": batch_size}
    
  return d

In [41]:
y_train_origin

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

Test

In [50]:
d = model(x_train, y_train_origin, x_test, y_test_origin, batch_size=4, lr=0.03,num_class=10)

Iteration number: 0, loss: 3.5455
Iteration number: 1, loss: 5.6579
Iteration number: 2, loss: 7.3833
Iteration number: 3, loss: 4.213

train accuracy: 16.57 %
test accuracy: 11.85 %
